<a href="https://colab.research.google.com/github/wmartinezf/Algoritmos-Informados-y-No-informados/blob/main/AlgoritmosInformadoipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# coding: utf-8

# # Algoritmos de búsqueda informada

# Los algoritmos de búsqueda informada utilizan conocimiento del dominio del problema para guiar la búsqueda hacia la solución. Esto se logra a través del uso de una función _heurística_.

# ## Funciones heurísticas

# Una función heurística es una estimación del costo a la meta u objetivo desde cualquier estado del grafo de estados-acciones.

# ### El rompecabezas del 15

# En el rompecabezas del 15 una posible función heurística consiste en contar el número de fichas desordenadas a partir de la configuración inicial.
# A continuación incluimos el código del rompecabezas previamente visto.

# In[1]:


from abc import ABC, abstractmethod

class EstadoProblema:
    """
    La clase EstadoProblema es abstracta.
    Representa un estado o configuración del problema a resolver.
    
    Es una interfaz simplificada para utilizarse
    en los algoritmos de búsqueda del curso.
    
    Al definir un problema particular hay que implementar los métodos
    abstractos
    """
    
    @abstractmethod
    def expand():
        """
        :return: el conjunto de estados sucesores
        """
        pass
    
    @abstractmethod
    def get_depth():
        """
        :return: la profundidad del estado
        """
        pass
        
    @abstractmethod
    def get_parent():
        """
        :return: referencia al estado predecesor o padre
        """
        pass


# In[2]:


from functools import reduce
import random    

# La secuencia del 0 al 15
# 0 representará el espacio en blanco
seq = list(range(0,16))

# Cuatro posibles acciones para nuestro agente
# Mover una ficha en dirección: 
# izquierda (E), derecha (W), arriba (N), o abajo (S)
actions = ['E','W','N','S']

# Representaremos las configuraciones con bits
# Definimos algunas funciones útiles
# Recorre un bloque de 4 bits de unos a la posición i
x_mask = lambda i: 15<<(4*i)

# Extrae los cuatro bits que están en la posción i
# en la configuración c
# El rompecabezas tiene 16 posiciones (16X4 = 64 bits)
extract = lambda i,c: (c&(x_mask(i)))>>(4*i)

# Verifica si la posición z es la última columna
e_most = lambda z: (z%4)==3

# Verifica si la posición z es la primera columna
w_most = lambda z: (z%4)==0

# Verifica si la posición z es el primer renglón
n_most = lambda z: z<=3

# Verifica si la posición z es el último renglón
s_most = lambda z:z>=12

# Establecemos un diccionario con las acciones posibles
# para cada posición del rompecabezas
valid_moves = {i:list(filter(lambda action:((not action=='E') or (not e_most(i))) and ((not action=='W') or (not w_most(i))) and ((not action=='S') or (not s_most(i))) and ((not action=='N') or (not n_most(i))),actions)) for i in seq}

# Realiza el movimiento hacía la izquierda
def move_east(puzzle):
    """
    :param puzzle: el rompecabezas
    """
    if(not e_most(puzzle.zero)):
        puzzle.zero += 1;
        mask = x_mask(puzzle.zero)
        puzzle.configuration =         (puzzle.configuration&mask)>>4 |         (puzzle.configuration&~mask)

# Realiza el movimiento hacía la derecha
def move_west(puzzle):
    if(not w_most(puzzle.zero)):
        puzzle.zero -= 1;
        mask = x_mask(puzzle.zero)
        puzzle.configuration =         (puzzle.configuration&mask)<<4 |         (puzzle.configuration&~mask)

# Realiza el movimiento hacía arriba
def move_north(puzzle):
    if(not n_most(puzzle.zero)):
        puzzle.zero -= 4;
        mask = x_mask(puzzle.zero)
        puzzle.configuration =         (puzzle.configuration&mask)<<16 |         (puzzle.configuration&~mask)

# Realiza el movimiento hacía abajo
def move_south(puzzle):
    if(not s_most(puzzle.zero)):
        puzzle.zero += 4;
        mask = x_mask(puzzle.zero)
        puzzle.configuration =         (puzzle.configuration&mask)>>16 |         (puzzle.configuration&~mask)

class Puzzle(EstadoProblema):
    """
    Rompecabezas del 15
    """
    
    
    def __init__(self, parent=None, action =None, depth=0):
        """
        Puede crearse un rompecabezas ordenado al no especificar
        parámetros del constructor.
        También se puede crear una nueva configuración a 
        partir de una configuración dada en parent.
        :param parent: configuración de referencia.
        :param action: la acción que se aplica a parent para
        generar la configuración sucesora.
        :depth la profundidad del estado a crear
        """
        self.parent = parent
        self.depth = depth
        if(parent == None):
            self.configuration =                  reduce(lambda x,y: x | (y << 4*(y-1)), seq)
            # posición del cero
            self.zero = 15
        else:
            self.configuration = parent.configuration
            self.zero = parent.zero
            if(action != None):
                self.move(action)

    def __str__(self):
        """
        :return: un string que representa 
        la configuración del rompecabezas
        """
        return '\n'+''.join(list(map(lambda i:        format(extract(i,self.configuration)," x")+        ('\n' if (i+1)%4==0 else ''),seq)))+'\n'

    def __repr__(self):
        """
        :return: representación texto de la configuración
        """
        return self.__str__()

    def __eq__(self,other):
        """
        :param other: la otra configuración con la que se comparará
        el objeto
        :return: verdadero cuando el objeto y el parámetro
        tienen la misma configuración.
        """
        return (isinstance(other, self.__class__)) and         (self.configuration==other.configuration)

    def __ne__(self,other):
        """
        :param other: la otra configuración con la que se comparará
        el objeto
        :return: verdadero cuando el objeto y el parámetro
        no tienen la misma configuración
        """
        return not self.__eq__(other)
        
    def __lt__(self,other):
        """
        :param other: la otra configuración con la que se comparará
        el objeto
        :return: verdadero cuando la profundidad del objeto
        es menor que la del argumento
        """
        return self.depth < other.depth

    def __hash__(self):
        """
        :return: un número hash para poder usar la configuración en 
        un diccionario, delegamos al hash de un entero
        """
        return hash(self.configuration)

    def move(self,action):
        """
        Realiza un movimiento de ficha.
        Debemos imaginar que el espacio se mueve en la dirección
        especificada por acción
        :param action: la acción a realizar
        """
        if(action =='E'):
            move_east(self)
        if(action =='W'):
            move_west(self)
        if(action =='N'):
            move_north(self)
        if(action =='S'):
            move_south(self)
        return self


    @staticmethod
    def to_list(puzzle):
        """
        Convertimos la configuración a una lista de números
        :param puzzle: la configuración a convertir
        :return la lista con enteros
        """
        return [extract(i,puzzle.configuration) for i in seq]

    def shuffle(self,n):
        """
        Desordena de manera aleatoria el rompecabezas.
        :param n: el número de movimientos aleatorios a aplicar
        """
        for i in range(0,n):
            self.move(random.choice(valid_moves[self.zero]))
        return self

    def expand(self):
        """
        Los sucesores del estado, quitamos el estado padre
        """
        #filtering the path back to parent
        return list(filter(lambda x:         (x!=self.parent),         [Puzzle(self,action,self.depth+1)         for action in valid_moves[self.zero]]))
    
    def get_depth(self):
        """
        :return: la profundidad del estado
        """
        return self.depth
    
    def get_parent(self):
        """
        :return: el nodo predecesor (padre) del estado 
        """
        return self.parent


# La clase __Puzzle__ incluye un método que nos hará muy simple el poder determinar el número de finchas fuera de lugar. 
# El método se llama __to_list__.

# In[3]:


# Ejemplo del método to_list
# Creamos un rompecabezas ordenado
ordenado = Puzzle()
print("rompecabezas ordenado:",ordenado)
lista_ordenada = Puzzle.to_list(ordenado)
print("el rompecabezas ordenado como una lista:\n",lista_ordenada)


# In[4]:


# Si desordenamos el rompecabezas su lista ya no estará ordenada
from random import seed
seed(2019)
desordenado = Puzzle()
desordenado.shuffle(5)
print("rompecabezas desordenado:",desordenado)
lista_desordenada = Puzzle.to_list(desordenado)
print("el rompecabezas desordenado como una lista:\n",lista_desordenada)


# Definamos la heurística __h1__ como el número de fichas fuera de lugar.

# In[5]:


def h1(p_1,p_2):
    # cuenta el número de fichas que no están en orden
    return sum(1     for a,b in zip(Puzzle.to_list(p_1),Puzzle.to_list(p_2)) if a!=b)


# In[6]:


print("ordenado:",ordenado)
print("desordenado:",desordenado)
print("número de fichas fuera de lugar:",h1(ordenado,desordenado))


In [ ]:
# ## Algoritmo A*

# El algorimo A\* utiliza una función heurística para priorizar la expansión de los estados.
# Primero nuestro método para recuperar la ruta a partir de un estado.

# In[7]:


from collections import deque

# trajectory nos regresará la trayectoria a partir de un estado
def trajectory(end):
    # nos valemos de un deque para almacenar la ruta
    sequence = deque()
    # agregamos el estado final o meta
    sequence.append(end)
    # nos vamos regresando al estado predecesor mientras este exista
    while end.get_parent():
        # nos movemos al predecesor
        end = end.get_parent()
        # lo agregamos a la lista
        sequence.append(end)
    # invertimos el orden de la secuencia
    sequence.reverse()
    # lo regresamos como una lista
    return list(sequence)


# A continuación mostramos una implementación de A* basada en una cola de prioridad.

# In[8]:


import heapq

class AStar:
    """
    Implementación del algoritmo A*
    """
    @staticmethod
    def search(origen,stop,g,h):
        """
        Búsqueda informada A*
        :param origen: estado inicial
        :param stop: función de paro, verdadera para el estado meta
        :param g: función de costo acumulado
        :param h: función heurística, costo estimado a la meta
        """
        # Nuestra cola de prioridad
        agenda = []
        # Conjunto de estados expandidos
        expandidos = set()
        # Condición trivial
        if stop(origen):
            return trajectory(origen)
        
        # Estado inicial a la cola de prioridad
        # La prioridad será f(s) = g(s) + h(s), 
        # para s una configuración
        f = lambda s: g(s) + h(s)
        
        # Agregamos el origen a la agenda
        heapq.heappush(agenda,(f(origen),origen))
        
        # Mientras la agenda no este vacía
        while agenda:
            # El frente de la cola de prioridad es la configuración
            # de menor costo f
            nodo = heapq.heappop(agenda)[1]
            # Agregamos el estado a la lista de expandidos
            expandidos.add(nodo)
            # En A* es necesario verificar la condición de 
            # paro tras sacar el elemento de la agenda
            if stop(nodo):
                return trajectory(nodo)
            # Realizamos la expansión del vértice
            for sucesor in nodo.expand():
                # Agregamos a la cola de prioridad siempre que no se haya
                # expandido previamente
                if sucesor not in expandidos:
                    heapq.heappush(agenda,(f(sucesor),sucesor))
        # No hay ruta al nodo meta
        # instrucción redundante
        return None


In [ ]:
# A continuación vamos a probar el algoritmo.


seed(650)
p = Puzzle()
# 40 movimientos aleatorios
p.shuffle(30)
print("rompecabezas a resolver:",p)
ruta = AStar.search(p, # rompecabezas desordenado
                    lambda s:s==Puzzle(), # detenerse si esta ordenado
                    lambda s:s.get_depth(), # el costo acumulado es la profunidad
                    lambda s:h1(s,Puzzle())) # la heurística h1
print("la ruta encontrada:",ruta)
print("longitud de la ruta:",len(ruta)-1)

In [ ]:
h_ucs = lambda s: 0

# Invocamos al algoritmo A*, se comportará como UCS
ruta = AStar.search(p, # rompecabezas desordenado
                    lambda s:s==Puzzl
                    e(), # detenerse si esta ordenado
                    lambda s:s.get_depth(), # el costo acumulado es la profunidad
                    h_ucs) # la heurística es cero para toda configuración
print("la ruta encontrada:",ruta)
print("longitud de la ruta:",len(ruta)-1)

In [ ]:
# Para obtener el algortimo GBFS no tomamos
# en cuenta el costo acumulado
# solo nos fijamos en la estimación a la meta
# para ello hacemos la función de costo igual a cero
g_gbfs = lambda s:0

# Invocamos al algoritmo A*, se comportará como GBFS
ruta = AStar.search(p, # rompecabezas desordenado
                    lambda s:s==Puzzle(), # detenerse si esta ordenado

                    g_gbfs, # el costo acumulado no se tomará en cuenta
                    lambda s:h1(s,Puzzle())) # la heurística h1
print("la ruta encontrada:",ruta)
print("longitud de la ruta:",len(ruta)-1)